In [1]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [2]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [3]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-validation").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

122 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/drop-musique-mini',
   'name': 'default',
   'split': 'train'},
  'qa': {'model': 'llama-3-8b',
   'temperature': 0.0,
   'top_p': 0.999,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.37,
  'f1': 0.5993370884585718,
  'fuzzy_match': 0.41,
  '0hops': {'exact_match': 0.37,
   'f1': 0.5993370884585718,
   'fuzzy_match': 0.41},
  'gen_token_count': {'all': {'count': 100.0,
    'mean': 6.41,
    'std': 4.431510219810265,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 28.0},
   'success': {'count': 65.0,
    'mean': 5.3076923076923075,
    'std': 2.8718628368789054,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 5.0,
    'max': 22.0},
   'fail': {'count': 35.0,
    'mean': 8.457142857142857,
    'std

In [4]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

122 experiments before preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.999,direct/format-few.txt,...,5.0,22.0,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0
1,9383a2592f931d3ab62f12bd9e0762a4afe403fb,951e866aa22f882a955257af5cf28abfa3eb8c92,tutti-rale,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,1648.0,5050.0,42.0,2030.214286,1835.654300,848.0,1293.50,1517.5,1900.00,10683.0
2,9383a2592f931d3ab62f12bd9e0762a4afe403fb,5ff24fc9d317f410a57700aadb91872053ddae9e,zippy-ears,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,1652.5,3196.0,41.0,1651.341463,828.334733,900.0,1195.00,1365.0,1778.00,5112.0
3,9383a2592f931d3ab62f12bd9e0762a4afe403fb,5b4f051756fedcb4397a68871b99ea72fa06c8da,peart-he'd,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,333.0,1404.0,47.0,356.148936,364.003164,112.0,201.50,311.0,384.50,2657.0
4,9383a2592f931d3ab62f12bd9e0762a4afe403fb,a5d52619eb00df6562c3cb5f423ae65cf60a3db2,ocher-body,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,369.0,645.0,40.0,321.675000,153.708580,137.0,228.75,264.5,377.75,829.0


In [5]:
if 'params.qa.top_p' not in df.columns:
    df['params.qa.top_p'] = 0.999

df['params.qa.top_p'] = df['params.qa.top_p'].map(lambda x: 0.999 if x > 0.99 else x)

In [6]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [7]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
print(f"{len(df)} experiments after dropping nan")

122 experiments after dropping nan


In [8]:
df.drop_duplicates(subset=param_cols + metric_cols[:2], keep='last', inplace=True)
print(f"{len(df)} experiments after dropping duplicates")

121 experiments after dropping duplicates


In [9]:
df.head()

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
1,9383a2592f931d3ab62f12bd9e0762a4afe403fb,951e866aa22f882a955257af5cf28abfa3eb8c92,tutti-rale,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,1648.0,5050.0,42.0,2030.214286,1835.654300,848.0,1293.50,1517.5,1900.00,10683.0
2,9383a2592f931d3ab62f12bd9e0762a4afe403fb,5ff24fc9d317f410a57700aadb91872053ddae9e,zippy-ears,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,1652.5,3196.0,41.0,1651.341463,828.334733,900.0,1195.00,1365.0,1778.00,5112.0
3,9383a2592f931d3ab62f12bd9e0762a4afe403fb,5b4f051756fedcb4397a68871b99ea72fa06c8da,peart-he'd,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,333.0,1404.0,47.0,356.148936,364.003164,112.0,201.50,311.0,384.50,2657.0
4,9383a2592f931d3ab62f12bd9e0762a4afe403fb,a5d52619eb00df6562c3cb5f423ae65cf60a3db2,ocher-body,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,369.0,645.0,40.0,321.675000,153.708580,137.0,228.75,264.5,377.75,829.0
5,9383a2592f931d3ab62f12bd9e0762a4afe403fb,9c2204a6ad086ba01011ce6f770c8af2cd00693f,tonal-cogs,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,322.0,727.0,41.0,452.756098,539.681007,160.0,222.00,281.0,381.00,2738.0


In [10]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [11]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

if len(df):
    df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [12]:
# make temperature 1 decimal
if len(df):
    df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [13]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/drop-musique-mini']

- params.dataset.name: ['default']

- params.dataset.split: ['validation', 'train']

- params.qa.model: ['deepseek-r1-llama-70b', 'llama-3-8b', 'llama-3.3-70b']

- params.qa.temperature: [0.5, 0.0]

- params.qa.top_p: [0.95, 0.999]

- params.qa.system_prompt: ['empty.txt', 'direct/format-few.txt', 'ccot/format-thought-drop.txt', 'direct/format-few-drop.txt']

- params.qa.user_prompt_template: ['icq-format-drop.txt', 'cq.txt']

- params.qa.few_shot_examples: ['drop/reasoning.json', 'manual/direct.json', 'drop/ccot.json', 'drop/direct.json']

- params.qa.n_shot: [0, 3, 1, 5]

- params.qa.n_sc: [5, 1]

- params.run: [1, 2, 3]



In [14]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [15]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [16]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [17]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.name": ["default"],
    "params.dataset.split": ["validation"],
    "params.qa.n_sc": [
        1,
        5,
    ],
    "params.qa.n_shot": [
        0,
        1,
        3,
        5,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [18]:
varying_params_list = [
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [0.0],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.0,
        ],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            "deepseek-r1-llama-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["empty.txt"],
        "params.qa.user_prompt_template": ["icq-format-drop.txt"],
        "params.qa.few_shot_examples": ["drop/reasoning.json"],
    },
]

In [19]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

120 experiment configurations
['params.dataset.path', 'params.dataset.name', 'params.dataset.split', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.temperature', 'params.qa.top_p', 'params.qa.system_prompt', 'params.qa.user_prompt_template', 'params.qa.few_shot_examples']


In [20]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 121


In [21]:
pairs = []
for exp_config in exp_configs:
    for existing_config in existing_configs:
        if len(sorted(exp_config.items())) != len(sorted(existing_config.items())):
            pairs.append((
                sorted(exp_config.items()), 
                sorted(existing_config.items())
            ))
assert len(pairs) == 0, pairs

In [22]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

0 missing configurations


In [23]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        if isinstance(arg_value, str) and '[' in arg_value:
            arg_value = f'"{arg_value}"'
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [24]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [25]:
df

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.technique,params.qa.instruction
1,9383a2592f931d3ab62f12bd9e0762a4afe403fb,951e866aa22f882a955257af5cf28abfa3eb8c92,tutti-rale,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,42.0,2030.214286,1835.654300,848.0,1293.50,1517.5,1900.00,10683.0,REASONING-SC(5),empty:icq-format-drop:drop/reasoning
2,9383a2592f931d3ab62f12bd9e0762a4afe403fb,5ff24fc9d317f410a57700aadb91872053ddae9e,zippy-ears,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,41.0,1651.341463,828.334733,900.0,1195.00,1365.0,1778.00,5112.0,REASONING-SC(5),empty:icq-format-drop:drop/reasoning
3,9383a2592f931d3ab62f12bd9e0762a4afe403fb,5b4f051756fedcb4397a68871b99ea72fa06c8da,peart-he'd,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,47.0,356.148936,364.003164,112.0,201.50,311.0,384.50,2657.0,REASONING,empty:icq-format-drop:drop/reasoning
4,9383a2592f931d3ab62f12bd9e0762a4afe403fb,a5d52619eb00df6562c3cb5f423ae65cf60a3db2,ocher-body,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,40.0,321.675000,153.708580,137.0,228.75,264.5,377.75,829.0,REASONING,empty:icq-format-drop:drop/reasoning
5,9383a2592f931d3ab62f12bd9e0762a4afe403fb,9c2204a6ad086ba01011ce6f770c8af2cd00693f,tonal-cogs,bdsaglam/drop-musique-mini,default,validation,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,41.0,452.756098,539.681007,160.0,222.00,281.0,381.00,2738.0,REASONING,empty:icq-format-drop:drop/reasoning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,cb4b874538a73c5774648d414aaddcf69cac56cb,94bb530df01a366469600961ba7f192e4a5161ee,glial-ruby,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,50.0,3.240000,2.272821,1.0,2.00,2.0,4.00,10.0,DIRECT,direct/format-few-drop:cq:drop/direct
118,cb4b874538a73c5774648d414aaddcf69cac56cb,3ecfab32d7cd35d648222a7570d39f292085e4aa,acred-tors,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.5,0.950,ccot/format-thought-drop.txt,...,36.0,42.277778,14.149429,25.0,33.50,42.0,47.00,105.0,CCOT,ccot/format-thought-drop:cq:drop/ccot
119,cb4b874538a73c5774648d414aaddcf69cac56cb,c8ba399869f8e1e0035312c10eb6ad427b25e020,minus-flus,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.5,0.950,direct/format-few-drop.txt,...,54.0,4.814815,9.514783,1.0,2.00,2.0,4.00,55.0,DIRECT,direct/format-few-drop:cq:drop/direct
120,cb4b874538a73c5774648d414aaddcf69cac56cb,c754f545eb6239c7a103ad4067ebfa8bda7ccfb9,balmy-ados,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,54.0,23.240741,32.432185,9.0,14.00,14.0,20.00,244.0,DIRECT-SC(5),direct/format-few-drop:cq:drop/direct
